In [32]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('C:\\Users\\Eduardo\\Documents\\Projects\\AutoTelos\\new_telos\\skills_with_levels.csv')

# Print the first few rows of the DataFrame
print(df.head())

# print the element 'learners' of the first row
print(df.loc[0, 'learners'])

         skill                                           learners
0       Cleave     Hermes, Nekomata, Chimera, Archangel, Valkyrie
1  Power Slash                      Valkyrie (15), Gurr, Rakshasa
2     Getsu-ei  Rakshasa (18), Ares (24), Take-Mikazuchi (27),...
3       Zan-ei  Oberon (18), Ares, Titan, Pale Rider, Power (2...
4   Gale Slash   Rakshasa (22), Ghoul (23), Shiisaa, Flauros, Ose
Hermes, Nekomata, Chimera, Archangel, Valkyrie


this script associates each persona with their learnable skills and the level they learn that skill

In [33]:
import pandas as pd
import re

# Function to extract level and append it to skill
def append_level(skill, learner):
    match = re.search(r'(.+?)\s*\((\d+)\)', learner)
    if match:
        persona, level = match.groups()
        return f"{persona.strip()}|{skill}({level})"
    else:
        return f"{learner.strip()}|{skill}"

# Apply the function and explode
df['learners'] = df.apply(lambda x: [append_level(x['skill'], learner) for learner in x['learners'].split(', ')], axis=1)
df = df.explode('learners')

# Split the combined column into 'persona' and 'skill'
df[['persona', 'skill']] = df['learners'].str.split('|', expand=True)

# Drop the original 'learners' column and clean persona names
df.drop(columns='learners', inplace=True)
df['persona'] = df['persona'].str.lower()

# Correct specific persona names
df['persona'] = df['persona'].replace({'seiryu': 'seiryuu', 'susanoo': 'susano-o'})

# Group by 'persona' and aggregate 'skill' into a list
df = df.groupby('persona')['skill'].agg(list).reset_index()

# Print the resulting DataFrame
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df)


             persona                                              skill
0            abaddon  [Weary Thrust, Evil Smile, High Counter, Null ...
1              alice  [Die for Me!(59), Spirit Drain, Evil Touch, Ma...
2             alilat  [Mabufudyne, Makarakarn, Regenerate 3, Repel P...
3                alp  [Garu, Mudo(11), Marin Karin(7), Charmdi(8), D...
4             ananta  [Weary Thrust(76), Samarecarm(79), Mediarahan,...
5              angel    [Garu, Hama(7), Patra, Charmdi(8), Sukukaja(5)]
6             anubis  [Mahamaon, Mamudoon, Makarakarn(61), Null Elec...
7            apsaras                    [Bufu, Posumudi(4), Sukunda(5)]
8         ara mitama  [Bash, Agi, Rakukaja, Rakunda(7), Sukukaja(8),...
9          arahabaki  [Vicious Strike, Virus Breath, Tetrakarn(64), ...
10         archangel  [Cleave, Holy Arrow(12), Mazio(13), Hama, Patr...
11              ares  [Getsu-ei(24), Zan-ei, Rakunda, Rebellion(22),...
12             asura  [Akasha Arts, Agidyne, Megidolaon(88), Tet

In [34]:
import pandas as pd

# Load the text file into a DataFrame
df_site = pd.read_csv('C:\\Users\\Eduardo\\Documents\\Projects\\AutoTelos\\fusion_charter\\site_compare.txt', sep='\t', header=None, names=['level', 'name', 'arcana'])

#make it so all persona names are in lowercase
df_site['name'] = df_site['name'].str.lower()

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df_site)

     level             name      arcana
0       68          abaddon       Devil
1       56            alice       Death
2       84           alilat     Empress
3        6              alp      Lovers
4       75           ananta        Aeon
5        4            angel     Justice
6       59           anubis    Judgment
7        3          apsaras   Priestess
8        6       ara mitama     Chariot
9       60        arahabaki      Hermit
10      10        archangel     Justice
11      19             ares     Chariot
12      85            asura         Sun
13      80          atavaka        Aeon
14      54          atropos     Fortune
15      67            attis  Hanged Man
16      71       baal zebul        Moon
17      52           barong     Emperor
18      81        beelzebub       Devil
19      13           berith  Hierophant
20      60      bishamonten       Tower
21      34      black frost        Fool
22      57           byakko  Temperance
23      58        chernobog        Moon


In [35]:
# Find the personas in df that are not in df_site
missing_in_df_site = df[~df['persona'].isin(df_site['name'])]

# Find the personas in df_site that are not in df
missing_in_df = df_site[~df_site['name'].isin(df['persona'])]

# Print the personas
print("Personas missing in df_site:")
print(missing_in_df_site['persona'])
print("\nPersonas missing in df:")
print(missing_in_df['name'])

Personas missing in df_site:
23          castor
24        cerberus
53          hermes
60              io
84           lucia
103        nemesis
118      palladion
121    penthesilea
123     polydeuces
Name: persona, dtype: object

Personas missing in df:
Series([], Name: name, dtype: object)


In [36]:
#remove from df all personas that are not in df_site
df = df[df['persona'].isin(df_site['name'])]

#print len of df
print(len(df))

169


In [37]:
df.to_csv('inheritable_skills_with_levels.csv', index=False)